# Lectura de ficheros

En este notebook presentamos los ejemplos del capítulo 1 del libro *Big Data con Python: recolección, almacenamiento y procesamiento de datos*. En este capítulo aprendemos cómo cargar en Python aquellos datos que tenemos ya disponibles en distintos formatos. Los datos usados en esta capítulo, disponibles en la carpeta *Cap1* de GitHub, fueron descargados de la [iniciativa de datos abiertos del Gobierno de España](http://datos.gob.es/) en marzo de 2018. Los ficheros en CSV, JSON, XML y XLS se refieren a las subvenciones asignadas en 2016 a asociaciones del ámbito educativo por el ayuntamiento de Alcobendas; por su parte, el fichero TSV lo crearemos a partir del CSV.

## CSV

El primero de los formatos que veremos es CSV. Importamos la biblioteca correspondiente, **csv**

In [28]:
import csv

Y comenzamos la carga del fichero. En primer lugar cargamos como un objeto iterable y lo recorremos con un bucle **for** para calcular el importe total de las subvenciones:

In [29]:
with open('../../data/Cap1/subvenciones.csv') as fichero_csv:
    lector = csv.reader(fichero_csv)
    next(lector, None)  # Se salta la cabecera
    importe_total = 0
    for linea in lector:
        importe_str = linea[2]
        importe = float(importe_str)
        importe_total = importe_total + importe
    print(importe_total)    

66487.94


También podemos calcular las subvenciones por centro almacenando un diccionario que tenga como clave el nombre del centro y como valor las subvenciones recibidas:

In [0]:
with open('../../data/Cap1/subvenciones.csv', encoding='latin1') as fichero_csv:
    lector = csv.reader(fichero_csv)
    next(lector, None)
    asociaciones = {}
    for linea in lector:
        centro = linea[0]
        subvencion = float(linea[2])
        if centro in asociaciones:
            asociaciones[centro] = asociaciones[centro] + subvencion
        else:
            asociaciones[centro] = subvencion
    print(asociaciones)

Es también posible cargar el fichero como un diccionario. En este caso cada una de las filas será un diccionario en el que la clave será el nombre dado en la cabecera del fichero y el valor el indicado en esa fila. De esta manera podemos acceder a los valores usando nombres y no la posición, lo que nos permite escribir un código más intuitivo. A continuación mostramos cómo sería el código para calcular el diccionario de subvenciones mostrado anteriormente de esta manera:

In [0]:
with open('../../data/Cap1/subvenciones.csv', encoding='latin1') as fichero_csv:
    dict_lector = csv.DictReader(fichero_csv)
    asociaciones = {}
    for linea in dict_lector:
        centro = linea['Asociación']
        subvencion = float(linea['Importe'])
        if centro in asociaciones:
            asociaciones[centro] = asociaciones[centro] + subvencion
        else:
            asociaciones[centro] = subvencion
    print(asociaciones)

Es interesante notar como en el código anterior no fue necesario saltarse la primera línea del iterador, ya que se consume directamente cuando cargamos el fichero como un diccionario para asignar nombres a los campos. Si la primera fila no tuviese estos nombres podríamos usar la opción **fieldnames=[..]** para especificarlos y sí se cargaría. Ahora vamos a ver cómo crear un nuevo fichero CSV. Para ello, vamos a modificar nuestro fichero de subvenciones con dos nuevas columnas, **"Justificación requerida"** y **"Justificación recibida"**. En el primera almacenaremos **Sí** si la subvención pasa de **300** euros y **No** en otro caso; en la segunda pondremos siempre **No**, ya que todavía no hemos recibido justificación alguna.

In [0]:
with open('../../data/Cap1/subvenciones.csv', encoding='latin1') as fich_lect, open('../../data/Cap1/subvenciones_esc.csv', 'w', encoding='latin1') as fich_escr:
    dict_lector = csv.DictReader(fich_lect)
    campos = dict_lector.fieldnames + ['Justificación requerida', 'Justificación recibida']
    escritor = csv.DictWriter(fich_escr, fieldnames=campos)
    escritor.writeheader()
    for linea in dict_lector:
        if float(linea['Importe']) > 300:
            linea['Justificación requerida'] = "Sí"
        else:
            linea['Justificación requerida'] = "No"
        linea['Justificación recibida'] = "No"
        escritor.writerow(linea)

## TSV

El formato TSV es similar al CSV, pero en el caso de TSV las columnas se separan con tabuladores. Para crear y manipular estos ficheros seguiremos usando la biblioteca **csv**, usando la opción **delimiter='\t'**. Por ejemplo, podemos crear un fichero TSV a partir del CSV anterior simplemente usando esta opción al crear el objeto **escritor**:

In [0]:
with open('../../data/Cap1/subvenciones.csv', encoding='latin1') as fich_lect, open('../../data/Cap1/subvenciones.tsv', 'w', encoding='latin1') as fich_escr:
    dict_lector = csv.DictReader(fich_lect)
    campos = dict_lector.fieldnames
    escritor = csv.DictWriter(fich_escr, delimiter='\t', fieldnames=campos)
    escritor.writeheader()
    for linea in dict_lector:
        escritor.writerow(linea)

Una vez creado el fichero, podemos recorrerlo como hacíamos arriba, eligiendo como separador el tabulador. La función siguiente calcula las subvenciones por centro como hicimos con CSV:

In [0]:
with open('../../data/Cap1/subvenciones.tsv', encoding='latin1') as fichero:
    dict_lector = csv.DictReader(fichero, delimiter='\t')
    asociaciones = {}
    for linea in dict_lector:
        centro = linea['Asociación']
        subvencion = float(linea['Importe'])
        if centro in asociaciones:
            asociaciones[centro] = asociaciones[centro] + subvencion
        else:
            asociaciones[centro] = subvencion
    print(asociaciones)

## JSON

En esta sección cargaremos los ficheros de subvenciones dados en formato JSON. En primer lugar, cargamos la biblioteca **json**

In [0]:
import json

Los objetos JSON se cargan en Python como diccionarios, mientras que los *arrays* JSON se traducen como listas Python. Con esta información podemos usar **load** para cargar y visualizar el fichero:

In [0]:
with open('../../data/Cap1/subvenciones.json') as fich:
    datos = json.load(fich)
    print(datos[0:2])

Como vemos, nuestro fichero JSON consiste en una lista de objetos donde cada uno de estos objetos se corresponde con una subvención, es decir, contiene 3 campos correspondientes a **"Asociación"**, **"Actividad Subvencionada"** e **"Importe en euros"**. Sin embargo, el disponer de un formato más flexible nos permite almacenar la información de manera más compacta: podríamos tener un objeto JSON por cada centro, con los campos **"Asociación"** y **"Actividades"**. Este segundo campo consistiría a su vez en una lista de objetos con los campos **"Actividad Subvencionada"** e **"Importe en euros"**. Vamos a escribir el código Python necesario para transformar nuestro fichero y guardarlo en un nuevo fichero **subvenciones_agrupadas.json**:

In [0]:
with open('../../data/Cap1/subvenciones.json', encoding='utf-8') as fich_lect, open('../../data/Cap1/subvenciones_agrupadas.json', 'w', encoding='utf-8') as fich_escr:
    data = json.load(fich_lect)
    asoc_str = "Asociación"
    act_str = "Actividad Subvencionada"
    imp_str = "Importe en euros"
    lista = []
    lista_act = []
    asoc_actual = ""
    dicc = {}
    for elem in data:
        asoc = elem[asoc_str]
        act = elem[act_str]
        imp = elem[imp_str]
        if asoc_actual != asoc:
            dicc["Actividades"] = lista_act
            dicc = {"Asociación": asoc}
            lista.append(dicc)
            lista_act = []
        lista_act.append({act_str : act, imp_str : imp})
        asoc_actual = asoc
    print(lista)
    json.dump(lista, fich_escr, ensure_ascii=False, indent=4) # , sort_keys=False

Intentemos ahora calcular, como hicimos para CSV, el total de gasto para cada centro y almacenarlo como un nuevo campo de la estructura que hemos creado arriba. El código necesario para ello es:

In [0]:
with open('../../data/Cap1/subvenciones.json', encoding='utf-8') as fich_lect, open('../../data/Cap1/subvenciones_agrupadas_error.json', 'w', encoding='utf-8') as fich_escr:
    data = json.load(fich_lect)
    asoc_str = "Asociación"
    act_str = "Actividad Subvencionada"
    imp_str = "Importe en euros"
    lista = []
    lista_act = []
    asoc_actual = ""
    dicc = {}
    gasto = 0
    for elem in data:
        asoc = elem[asoc_str]
        act = elem[act_str]
        imp = float(elem[imp_str])
        if asoc_actual != asoc:
            dicc["Actividades"] = lista_act
            dicc["Gasto"] = gasto
            dicc = {"Asociación": asoc}
            lista.append(dicc)
            lista_act = []
            gasto = 0
        lista_act.append({act_str : act, imp_str : imp})
        gasto = gasto + imp
        asoc_actual = asoc
    print(lista)
    json.dump(lista, fich_escr, ensure_ascii=False, indent=4) # , sort_keys=False

Como se puede observar, ¡el código falla! El problema es que se han añadido puntos en cantidades con miles de euros. Por ejemplo, 1000 euros se han introducido en el fichero JSON como 1.000.00, indicando el primer punto los miles y el segundo los decimales. Python no es capaz de analizar este número, por lo que vamos a crear el nuevo JSON a partir del fichero CSV que usamos en el capítulo anterior.

In [0]:
with open('../../data/Cap1/subvenciones.csv', encoding='latin1') as fich_lect, open('../../data/Cap1/subvenciones_agrupadas_con_gasto.json', 'w', encoding='utf-8') as fich_escr:
    dict_lector = csv.DictReader(fich_lect)
    asoc_str = "Asociación"
    act_str = "Actividad Subvencionada "
    imp_str = "Importe"
    lista = []
    lista_act = []
    asoc_actual = ""
    dicc = {}
    gasto = 0
    for linea in dict_lector:
        asoc = linea[asoc_str]
        act = linea[act_str]
        imp = float(linea[imp_str])
        if asoc_actual != asoc:
            dicc["Actividades"] = lista_act
            dicc["Gasto"] = gasto
            dicc = {"Asociación": asoc}
            lista.append(dicc)
            lista_act = []
            gasto = 0
        lista_act.append({act_str : act, imp_str : imp})
        gasto = gasto + imp
        asoc_actual = asoc
    json.dump(lista, fich_escr, ensure_ascii=False, indent=4) # , sort_keys=False

## Excel

Python no tiene una biblioteca estándar para tratar con ficheros Excel, por lo que presentaremos las bibliotecas **xlrd** y **xlwt** para lectura y escritura, respectivamente. Estas bibliotecas pueden ser consideradas estándares *de facto*, aunque presentaremos también brevemente la biblioteca **pandas** para lectura y escritura de dataframes desde/hacia ficheros Excel. Empezamos cargando las correspondientes bibliotecas:

In [0]:
from xlrd import open_workbook, colname
import xlwt

Empezaremos calculando la subvención total recibida por centro:

In [0]:
with open_workbook('../../data/Cap1/subvenciones.xls',on_demand=True) as libro:
    asociaciones = {}
    for nombre in libro.sheet_names():
        hoja = libro.sheet_by_name(nombre)
        for i in range(1,hoja.nrows):
            fila = hoja.row(i)
            centro = fila[0].value
            subvencion = fila[2].value
            #print(fila[0].ctype)
            #print(fila[2].value)
            if centro in asociaciones:
                asociaciones[centro] = asociaciones[centro] + subvencion
            else:
                asociaciones[centro] = subvencion
    print(asociaciones)

Ahora crearemos una nueva hoja, en la que tendremos una tabla con los centros, la subvención recibida, la subvención justificada, y la subvención que queda por justificar, que será una fórmula:

In [0]:
with open_workbook('../../data/Cap1/subvenciones.xls',on_demand=True) as libro_lect:
    asociaciones = {}
    libro_escr = xlwt.Workbook()
    for nombre in libro_lect.sheet_names():
        hoja_lect = libro_lect.sheet_by_name(nombre)
        hoja_escr = libro_escr.add_sheet(nombre)
        for i in range(hoja_lect.nrows):
            for j in range(hoja_lect.ncols):
                hoja_escr.write(i, j, hoja_lect.row(i)[j].value)
            if i != 0:
                fila = hoja_lect.row(i)
                centro = fila[0].value
                subvencion = float(fila[2].value)
                if centro in asociaciones:
                    asociaciones[centro] = asociaciones[centro] + subvencion
                else:
                    asociaciones[centro] = subvencion
    hoja_escr = libro_escr.add_sheet('Totales')
    hoja_escr.write(0, 0, "Asociación")
    hoja_escr.write(0, 1, "Importe total")
    hoja_escr.write(0, 2, "Importe justificado")
    hoja_escr.write(0, 3, "Restante")
    for i, clave in enumerate(asociaciones):
        fila = i + 1
        fila_form = i + 2
        hoja_escr.write(fila, 0, clave)
        hoja_escr.write(fila, 1, asociaciones[clave])
        hoja_escr.write(fila, 2, 0)
        hoja_escr.write(fila, 3, xlwt.Formula("C" + str(fila_form) + "-" + "B" + str(fila_form)))
    libro_escr.save('../../data/Cap1/subvenciones_totales.xls')

In [0]:
print(colname(2), colname(35))

Es también posible usar la biblioteca **pandas** para cargar y guardar **dataframes**:

In [0]:
import pandas
with pandas.ExcelFile('../../data/Cap1/subvenciones.xls') as xl:
    writer = pandas.ExcelWriter('../../data/Cap1/subvenciones_df.xls')
    for nombre in xl.sheet_names:
        df = xl.parse(nombre)
        df.to_excel(writer,nombre)
    writer.save()

## XML

Por último vamos a ver cómo manipular ficheros **XML** con la biblioteca **etree**.

In [0]:
import xml.etree.ElementTree as ET

Como en los ejemplos anteriores, vamos a empezar recorriendo el fichero y calculando un diccionario con la subvención total para cada centro:

In [0]:
arbol = ET.parse('../../data/Cap1/subvenciones.xml')
raiz = arbol.getroot()
asociaciones = {}
for fila in raiz:
    centro = fila[0].text
    subvencion = float(fila[2].text)
    if centro in asociaciones:
        asociaciones[centro] = asociaciones[centro] + subvencion
    else:
        asociaciones[centro] = subvencion
print(asociaciones)

In [0]:
arbol = ET.parse('../../data/Cap1/subvenciones.xml')
asociaciones = {}
for fila in arbol.findall('Row'): # raiz.iter('Row'):
    centro = fila.find('Asociaci_n').text
    subvencion = float(fila.find('Importe').text)
    if centro in asociaciones:
        asociaciones[centro] = asociaciones[centro] + subvencion
    else:
        asociaciones[centro] = subvencion
print(asociaciones)

Igual que ocurría con JSON, el formato XML es más flexible que CVS y Excel y nos permite representar la información de manera más compacta. Vamos a crear un nuevo fichero XML a partir del que tenemos que contará con una raíz que tendrá como elementos las distintas asociaciones. Cada **Asociacion** tendrá como atributo su **nombre** y como elementos la subvención **Total** y la lista de **Actividades**. La lista de actividades tendrá elemenos **Actividad** con **Nombre** y **Gasto**.

In [0]:
arbol = ET.parse('../../data/Cap1/subvenciones.xml')
raiz = arbol.getroot()
nuevo = ET.ElementTree()
raiz_nueva = ET.Element("Raiz")
nuevo._setroot(raiz_nueva)
elem_actual = ET.Element("Asociacion")
asoc_actual = ""
actividades = ET.SubElement(elem_actual, "Actividades")
gasto = 0
for fila in raiz.findall('Row'):
    asoc = fila.find('Asociaci_n').text
    act = fila.find('Actividad_Subvencionada').text
    imp = float(fila.find('Importe').text)
    if asoc_actual != asoc:
        gas_total = ET.SubElement(elem_actual, "Total")
        gas_total.text = str(gasto)
        elem_actual = ET.SubElement(raiz_nueva, "Asociacion")
        elem_actual.set('nombre', asoc)
        actividades = ET.SubElement(elem_actual, "Actividades")
        gasto = 0
    act_elem = ET.SubElement(actividades, "Actividad")
    nom_elem = ET.SubElement(act_elem, "Nombre")
    nom_elem.text = act
    imp_elem = ET.SubElement(act_elem, "Subvencion")
    imp_elem.text = str(imp)
    gasto = gasto + imp
    asoc_actual = asoc
nuevo.write('../../data/Cap1/subvenciones_lista_total.xml')